In [ ]:

"""
Beating the Benchmark by stacking 3 models
Search Results Relevance @ Kaggle
__author__ : Amit Sharma

"""
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition, pipeline, metrics, grid_search
from nltk.stem.porter import *
import pandas as pd
import numpy as np
import re
import string
from bs4 import BeautifulSoup
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import StratifiedShuffleSplit

pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60) 

In [ ]:
# Reading the trainTest data
trainAll = pd.read_csv('C:\\axs\\work\\kaggle\\crowdflower\\input\\train.csv')

# create labels. drop useless columns
yAll = trainAll.median_relevance.values
trainAll = trainAll.drop(['relevance_variance'], axis=1)

#Take a smaller percentage of train
sss = StratifiedShuffleSplit(yAll, n_iter=3, test_size=0.3)

for train_index,test_index in sss:
    trainMaster, testMaster = trainAll.iloc[train_index], trainAll.iloc[test_index]
    y, testY = yAll[train_index], yAll[test_index]

In [ ]:
#stopwords tweak - more overhead
sw=[]
stop_words = ['http','www','img','border','0','1','2','3','4','5','6','7','8','9']
stop_words = text.ENGLISH_STOP_WORDS.union(stop_words)
for stw in stop_words:
    sw.append("q"+stw)
    sw.append("z"+stw)
stop_words = text.ENGLISH_STOP_WORDS.union(sw)

In [ ]:
# Model 1 - TF IDF based
train = trainMaster.copy()
test = testMaster.copy()

train = train.drop(['id', 'median_relevance'], axis=1)
test = test.drop(['id', 'median_relevance'], axis=1)

# The following 3 functions have been taken from Ben Hamner's github repository
# https://github.com/benhamner/Metrics
def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)


if __name__ == '__main__':
    # do some lambda magic on text columns
    traindata = list(train.apply(lambda x:'%s %s' % (x['query'],x['product_title']),axis=1))
    testdata = list(test.apply(lambda x:'%s %s' % (x['query'],x['product_title']),axis=1))
    
    # the infamous tfidf vectorizer (Do you remember this one?)
    tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = stop_words)
    
    # Fit TFIDF
    tfv.fit(traindata)
    X =  tfv.transform(traindata) 
    X_test = tfv.transform(testdata)
    
    # Initialize SVD
    svd = TruncatedSVD()
    
    # Initialize the standard scaler 
    scl = StandardScaler()
    
    # We will use SVM here..
    svm_model = SVC()
    
    # Create the pipeline 
    clf = pipeline.Pipeline([('svd', svd),
    						 ('scl', scl),
                    	     ('svm', svm_model)])
    
    # Create a parameter grid to search for best parameters for everything in the pipeline
    param_grid = {'svd__n_components' : [400],
                  'svm__C': [10]}
    
    # Kappa Scorer 
    kappa_scorer = metrics.make_scorer(quadratic_weighted_kappa, greater_is_better = True)
    
    # Initialize Grid Search Model
    model = grid_search.GridSearchCV(estimator = clf, param_grid=param_grid, scoring=kappa_scorer,
                                     verbose=10, n_jobs=1, iid=True, refit=True, cv=2)
                                     
    # Fit Grid Search Model
    model.fit(X, y)
    print("Best score: %0.3f" % model.best_score_)
    print("Best parameters set:")
    best_parameters = model.best_estimator_.get_params()
    for param_name in sorted(param_grid.keys()):
    	print("\t%s: %r" % (param_name, best_parameters[param_name]))
    
    # Get best model
    best_model = model.best_estimator_
    
    # Fit model with best parameters optimized for quadratic_weighted_kappa
    best_model.fit(X,y)
    model1Preds = best_model.predict(X_test)
    #model1TrainPreds = best_model.predict(X)
    print "Model 1 kappa score: ", metrics.accuracy_score(testY, model1Preds.astype(int))

In [ ]:
# Model 2 - Using stemmer

train = trainMaster.copy().reset_index()
test = testMaster.copy().reset_index()

# array declarations
s_data = []
s_labels = []
t_data = []
t_labels = []

#remove html, remove non text or numeric, make query and title unique features for counts using prefix (accounted for in stopwords tweak)
stemmer = PorterStemmer()
## Stemming functionality
class stemmerUtility(object):
    """Stemming functionality"""
    @staticmethod
    def stemPorter(review_text):
        porter = PorterStemmer()
        preprocessed_docs = []
        for doc in review_text:
            final_doc = []
            for word in doc:
                final_doc.append(porter.stem(word))
                #final_doc.append(wordnet.lemmatize(word)) #note that lemmatize() can also takes part of speech as an argument!
            preprocessed_docs.append(final_doc)
        return preprocessed_docs


for i in range(len(train.id)):
    s=(" ").join(["q"+ z for z in BeautifulSoup(train["query"][i]).get_text(" ").split(" ")]) + \
        " " + (" ").join(["z"+ z for z in BeautifulSoup(train.product_title[i]).get_text(" ").split(" ")]) + \
        " " + BeautifulSoup(str(train.product_description[i])).get_text(" ")
    s=re.sub("[^a-zA-Z0-9]"," ", s)
    s= (" ").join([stemmer.stem(z) for z in s.split(" ")])
    s_data.append(s)
    s_labels.append(str(train["median_relevance"][i]))
for i in range(len(test.id)):
    s=(" ").join(["q"+ z for z in BeautifulSoup(test["query"][i]).get_text().split(" ")]) + \
        " " + (" ").join(["z"+ z for z in BeautifulSoup(test.product_title[i]).get_text().split(" ")]) + \
        " " + BeautifulSoup(str(test.product_description[i])).get_text()
    s=re.sub("[^a-zA-Z0-9]"," ", s)
    s= (" ").join([stemmer.stem(z) for z in s.split(" ")])
    t_data.append(s)
#create sklearn pipeline, fit all, and predit test data
clf = Pipeline([('v',TfidfVectorizer(min_df=5, max_df=500, max_features=None, strip_accents='unicode', 
                                     analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1, 2), use_idf=True,
                                     smooth_idf=True, sublinear_tf=True, stop_words = stop_words)), 
('svd', TruncatedSVD(n_components=200, algorithm='randomized', n_iter=5, random_state=None, tol=0.0)), 
('scl', StandardScaler(copy=True, with_mean=True, with_std=True)), 
('svm', SVC(C=10.0, kernel='rbf', degree=3, gamma=0.0, coef0=0.0, shrinking=True, probability=False, 
            tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, random_state=None))])
clf.fit(s_data, s_labels)
model2Preds = clf.predict(t_data).astype(int)
#model2TrainPreds = clf.predict(s_data)
print "Model 2 kappa score: ", metrics.accuracy_score(testY, model2Preds)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Model 2 - Using stemmer

train = trainMaster.copy()
test = testMaster.copy()

train = train.drop(['id', 'median_relevance'], axis=1)
test = test.drop(['id', 'median_relevance'], axis=1)

# Model 3 - Based on features
def query_title_common(row):
    return (set(row['query'].split()) & set(row['product_title'].split()))

def query_desc_common(row):
    return (set(row['query'].split()) & set(row['pdesc'].split()))


def getTitleFeatures(df):
    df['query'] = df['query'].astype('str')
    df['product_title'] = df['product_title'].astype('str')
    # Lowercase everything
    df['query'] = df['query'].str.lower()
    df['product_title'] = df['product_title'].str.lower()
    df['query_len'] = df['query'].str.len()
    df['ptitle_len'] = df['product_title'].str.len()

    df['query_nwords'] = df['query'].str.split().str.len()

    df['query_title'] = df.apply(query_title_common,axis=1)
    df['query_title_nwords'] = df['query_title'].str.len()
    df['title_nwords'] = df['product_title'].str.split().str.len()
    df['query_title_match'] = df['query_title_nwords']/df['query_nwords']
    
    #df = df[['query_nwords', 'title_nwords', 'query_title_nwords', 'query_title_match', 'query_len', 'ptitle_len']]
    df = df.drop(['query_title'],axis=1)
        
    return (df)

def getDescFeatures(df):
    df['product_description'] = df['product_description'].astype('str')
    df['product_description'] = df['product_description'].str.lower()
    df['pdesc'] = map(lambda x: BeautifulSoup(''.join(x)).text, df['product_description'])
    df['query_nwords'] = df['query'].str.split().str.len()

    df['query_desc'] = df.apply(query_desc_common,axis=1)
    df['query_desc_nwords'] = df['query_desc'].str.len()
    df['desc_nwords'] = df['pdesc'].str.split().str.len()
    df['query_desc_match'] = df['query_desc_nwords']/df['query_nwords']

    df['query'] = df['query'].astype(str)

    df['query_len'] = df['query'].str.len()
    df['pdesc_len'] = df['pdesc'].str.len()
    df['query_desc_match'] = (df['query_desc_nwords']+0.00001)/df['query_nwords']
    df['pdesc_size_over_query'] = df['pdesc_len']/df['query_len']
    
    df = df.drop(['pdesc', 'query_desc', ], axis=1)

    return (df)

def addFeatures(df):
    df = getTitleFeatures(df)
    df = getDescFeatures(df)
 
    # Create dummy columns for categorical variable as scikit does not understand factors
    #dummyDF = pd.get_dummies(df['query'])
    #dummyDF= dummyDF.astype(bool)
    #df = pd.concat([df,dummyDF],axis=1)
    
    df = df.drop(['query', 'product_title', 'product_description'], axis=1)

    return (df)

train = addFeatures(train)
test = addFeatures(test)

clf = GradientBoostingClassifier(n_estimators=25, learning_rate=0.7, max_depth=2, random_state=0).fit(train, y)

model3Preds = clf.predict(test)
#model3TrainPreds = clf.predict(train)
print "Model 3 kappa score: ", metrics.accuracy_score(testY, model3Preds.astype(int))

In [192]:
# Model 4

train = trainMaster.copy().reset_index()
test = testMaster.copy().reset_index()


# array declarations
sw=[]
s_data = []
s_labels = []
t_data = []
t_labels = []
stemmer = PorterStemmer()
#stopwords tweak - more overhead
stop_words = ['http','www','img','border','color','style','padding','table','font','thi','inch','ha','width','height',
'0','1','2','3','4','5','6','7','8','9']
#stop_words = text.ENGLISH_STOP_WORDS.union(stop_words)

#stop_words = ['http','www','img','border','0','1','2','3','4','5','6','7','8','9']
stop_words = text.ENGLISH_STOP_WORDS.union(stop_words)

punct = string.punctuation
punct_re = re.compile('[{}]'.format(re.escape(punct)))

#remove html, remove non text or numeric, make query and title unique features for counts using prefix (accounted for in stopwords tweak)
stemmer = PorterStemmer()
## Stemming functionality
class stemmerUtility(object):
    """Stemming functionality"""
    @staticmethod
    def stemPorter(review_text):
        porter = PorterStemmer()
        preprocessed_docs = []
        for doc in review_text:
            final_doc = []
            for word in doc:
                final_doc.append(porter.stem(word))
                #final_doc.append(wordnet.lemmatize(word)) #note that lemmatize() can also takes part of speech as an argument!
            preprocessed_docs.append(final_doc)
        return preprocessed_docs
    
def preprocess(x):
    x=x.lower()
    x=re.sub("[^a-zA-Z0-9]"," ", x)
    x=punct_re.sub(' ', x)
    new_x = []
    for token in x.split(' '):
        new_x.append(stemmer.stem(token))
    return ' '.join(new_x)
# Fit TFIDF
import scipy.sparse
def vectorize(train, tfv_query=None):
    query_data = list(train['query'].apply(preprocess))
    title_data = list(train['product_title'].apply(preprocess))
    if tfv_query is None:
        tfv_query = TfidfVectorizer(min_df=3,  max_features=None,   
                strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
                ngram_range=(1, 5), use_idf=1,smooth_idf=1,sublinear_tf=1,
                stop_words =stop_words)

        full_data = query_data + title_data
        tfv_query.fit(full_data)

    return scipy.sparse.hstack([tfv_query.transform(query_data), tfv_query.transform(title_data)]), tfv_query

X, tfv_query = vectorize(train)
X_test, _ = vectorize(test, tfv_query)

# Add title features
trainTitleFeatures = getTitleFeatures(train.copy())
trainTitleFeatures = trainTitleFeatures.drop(['index','id', 'query', 'median_relevance', 'product_title','product_description'], axis=1)
testTitleFeatures = getTitleFeatures(test.copy())
testTitleFeatures = testTitleFeatures.drop(['index','id', 'query', 'median_relevance', 'product_title','product_description'], axis=1)

trainDescFeatures = getDescFeatures(train.copy())
trainDescFeatures = trainDescFeatures.drop(['index','id', 'query', 'median_relevance', 'product_title','product_description'], axis=1)
testDescFeatures = getDescFeatures(test.copy())
testDescFeatures = testDescFeatures.drop(['index','id', 'query', 'median_relevance', 'product_title','product_description'], axis=1)


# Initialize SVD
svd = TruncatedSVD(n_components=400)
from sklearn.metrics.pairwise import linear_kernel
class FeatureInserter():

    def __init__(self):
        pass

    def transform(self, X, y=None):
        distances = []
        quasi_jaccard = []
        #print(len(distances), X.shape)
        
        titleFeatures = []
        descFeatures = []
        
        if X.shape[0] == trainTitleFeatures.shape[0]:
            titleFeatures = trainTitleFeatures
            descFeatures = trainDescFeatures
        else:
            titleFeatures = testTitleFeatures
            descFeatures = testDescFeatures

        for row in X.tocsr():
            row=row.toarray().ravel()
            #print row[:row.shape[0]/2], row[row.shape[0]/2:]
            cos_distance = linear_kernel(row[:row.shape[0]/2], row[row.shape[0]/2:])
            distances.append(cos_distance[0])
            intersect = row[:row.shape[0]/2].dot(row[row.shape[0]/2:])
            union = (row[:row.shape[0]/2]+row[row.shape[0]/2:]).dot((row[:row.shape[0]/2]+row[row.shape[0]/2:]))
            quasi_jaccard.append(1.0*intersect/union)

        #print(len(distances), X.shape)
        #print(distances[:10])

        #X = scipy.sparse.hstack([X, distances])
        #ret= np.matrix([x for x in zip(distances, quasi_jaccard)])
        ret = pd.DataFrame([x for x in zip(distances, quasi_jaccard)])
        ret = pd.concat([ret, titleFeatures], axis=1)
        #ret = pd.concat([ret, descFeatures], axis=1)
        
        return ret
    
    def fit(self, X,y):
        return self


    def fit_transform(self, X, y, **fit_params):
        self.fit(X,y)
        return self.transform(X)

# Initialize the standard scaler 
scl = StandardScaler()

# We will use SVM here..
svm_model = SVC(C=10.)

# Create the pipeline 
model = pipeline.Pipeline([('UnionInput', FeatureUnion([('svd', svd), ('dense_features', FeatureInserter())])),
                         ('scl', scl),
                         ('svm', svm_model)])
# Fit Model
model.fit(X, y)

preds = model.predict(X_test)

for i in range(len(train.id)):
    s=(" ").join(["q"+ z for z in BeautifulSoup(train["query"][i]).get_text(" ").split(" ")]) + \
        " " + (" ").join(["z"+ z for z in BeautifulSoup(train.product_title[i]).get_text(" ").split(" ")]) + \
        " " + BeautifulSoup(str(train.product_description[i])).get_text(" ")
    s=re.sub("[^a-zA-Z0-9]"," ", s)
    s= (" ").join([stemmer.stem(z) for z in s.split(" ")])
    s=s.lower()
    s=s.replace("nan", "")
    s_data.append(s)
    s_labels.append(str(train["median_relevance"][i]))
for i in range(len(test.id)):
    s=(" ").join(["q"+ z for z in BeautifulSoup(test["query"][i]).get_text().split(" ")]) + \
        " " + (" ").join(["z"+ z for z in BeautifulSoup(test.product_title[i]).get_text().split(" ")]) + \
        " " + BeautifulSoup(str(test.product_description[i])).get_text()
    s=re.sub("[^a-zA-Z0-9]"," ", s)
    s=s.lower()
    s=s.replace("nan", "")
    s= (" ").join([stemmer.stem(z) for z in s.split(" ")])
    t_data.append(s)
#create sklearn pipeline, fit all, and predit test data
clf = Pipeline([('v',TfidfVectorizer(min_df=5, max_df=500, max_features=None, strip_accents='unicode', \
                                    analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1, 5), use_idf=True, \
                                     smooth_idf=True, sublinear_tf=True, stop_words = stop_words)), 
('UnionInput', FeatureUnion([('svd', TruncatedSVD(n_components=300, algorithm='randomized', n_iter=5, random_state=None, tol=0.0)), ('dense_features', FeatureInserter())])),
#('svd', TruncatedSVD(n_components=300, algorithm='randomized', n_iter=5, random_state=None, tol=0.0)), 
('scl', StandardScaler(copy=True, with_mean=True, with_std=True)), 
('svm', SVC(C=10.0, kernel='rbf', degree=3, gamma=0.0, coef0=0.0, shrinking=True, probability=False, tol=0.001, \
            cache_size=200, class_weight=None, verbose=False, max_iter=-1, random_state=None))])
clf.fit(s_data, s_labels)
t_labels = clf.predict(t_data)

import math
model4Preds = []
for i in range(len(preds)):
    x =math.sqrt((int(t_labels[i]))*preds[i])
    x = round(x)
    model4Preds.append(int(x))
print "Model 4 kappa score: ", metrics.accuracy_score(testY, model4Preds)

Model 4 kappa score:  0.64501312336


In [194]:
metrics.accuracy_score(testY, t_labels.astype(int))

0.65485564304461941

In [ ]:
#Eval the results on the train set
print "Model 1 kappa score: ", metrics.accuracy_score(testY, model1Preds)
print "Model 2 kappa score: ", metrics.accuracy_score(testY, model2Preds)
print "Model 3 kappa score: ", metrics.accuracy_score(testY, model3Preds)
print "Model 4 kappa score: ", metrics.accuracy_score(testY, model4Preds)

In [ ]:
model1Preds.shape, model2Preds.shape
from scipy.stats.stats import pearsonr   
print "Correlation Coeff 1-2: ", pearsonr(model1Preds, model2Preds)
print "Correlation Coeff 1-3: ", pearsonr(model1Preds, model3Preds)
print "Correlation Coeff 1-4: ", pearsonr(model1Preds, model4Preds)
print "Correlation Coeff 2-3: ", pearsonr(model2Preds, model3Preds)
print "Correlation Coeff 2-4: ", pearsonr(model2Preds, model4Preds)
print "Correlation Coeff 3-4: ", pearsonr(model3Preds, model4Preds)

In [ ]:
import math
p12 = []
p13 = []
p14 = []
p23 = []
p24 = []
p34 = []

for i in range(len(testY)):
    x = math.sqrt(int(model1Preds[i]) * int(model2Preds[i]) )
    x = math.floor(x)
    p12.append(int(x))
    x = math.sqrt(int(model1Preds[i]) * int(model3Preds[i]) )
    x = math.floor(x)
    p13.append(int(x))
    x = math.sqrt(int(model1Preds[i]) * int(model4Preds[i]) )
    x = math.floor(x)
    p14.append(int(x))
    x = math.sqrt(int(model2Preds[i]) * int(model3Preds[i]) )
    x = math.floor(x)
    p23.append(int(x))
    x = math.sqrt(int(model2Preds[i]) * int(model4Preds[i]) )
    x = math.floor(x)
    p24.append(int(x))
    x = math.sqrt(int(model3Preds[i]) * int(model4Preds[i]) )
    x = math.floor(x)
    p34.append(int(x))
    
    
print "Blended kappa score p12: ", metrics.accuracy_score(testY, p12)    
print "Blended kappa score p13: ", metrics.accuracy_score(testY, p13)    
print "Blended kappa score p14: ", metrics.accuracy_score(testY, p14)    
print "Blended kappa score p23: ", metrics.accuracy_score(testY, p23)    
print "Blended kappa score p24: ", metrics.accuracy_score(testY, p24)    
print "Blended kappa score p34: ", metrics.accuracy_score(testY, p34)    